In [25]:
import cv2
import numpy as np

# Fonction pour détecter les obstacles noirs et la zone d'arrivée dans une image
def detecter_obstacles_et_arrivee(image_path):
    # Charger l'image
    image = cv2.imread(image_path)
    height, width, _ = image.shape  # Obtenir la hauteur et la largeur de l'image

    # Convertir l'image en espace de couleur HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Définir une plage de valeurs pour le rouge en HSV
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

    # Filtrer l'image pour ne conserver que les pixels rouges
    mask_red = cv2.inRange(hsv, lower_red, upper_red)

    # Appliquer un flou à la masque pour réduire le bruit
    blurred_mask_red = cv2.GaussianBlur(mask_red, (5, 5), 0)

    # Trouver les contours dans le masque filtré
    contours_red, _ = cv2.findContours(blurred_mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Liste pour stocker les coordonnées de la zone d'arrivée
    zone_arrivee_coords = []

    # Parcourir tous les contours rouges trouvés
    for contour in contours_red:
        # Ignorer les petits contours qui pourraient être du bruit
        if cv2.contourArea(contour) > 100:
            # Obtenir les coordonnées du rectangle englobant
            x, y, w, h = cv2.boundingRect(contour)

            # Ajouter les coordonnées à la liste de la zone d'arrivée
            zone_arrivee_coords.append([(x, height - y), (x + w, height - y), (x + w, height - y - h), (x, height - y - h)])

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Appliquer un flou pour réduire le bruit
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Appliquer la détection des bords avec Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Trouver les contours dans l'image des bords
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Liste pour stocker les coordonnées des obstacles
    obstacles_coords = []

    # Parcourir tous les contours trouvés
    for contour in contours:
        # Ignorer les petits contours qui pourraient être du bruit
        if cv2.contourArea(contour) > 100:
            # Obtenir les coordonnées des sommets du contour
            approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
            
            # Ajouter les coordonnées à la liste des obstacles
            obstacles_coords.append([tuple((x, height - y)) for x, y in approx.reshape(-1, 2)])

    # Dessiner les rectangles autour des obstacles détectés en vert
    # et autour de la zone d'arrivée en violet
    image_with_obstacles = image.copy()
    for obstacle in obstacles_coords:
        cv2.polylines(image_with_obstacles, [np.array(obstacle)], True, (0, 255, 0), 2)

    for zone_arrivee in zone_arrivee_coords:
        cv2.polylines(image_with_obstacles, [np.array(zone_arrivee)], True, (255, 0, 255), 2)

    # Afficher l'image avec les polygones verts et violets
    cv2.imshow('Image avec obstacles et zone d\'arrivée', image_with_obstacles)
    
    # Attendre 1 seconde (1000 millisecondes) et quitter si une touche est pressée
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return obstacles_coords, zone_arrivee_coords

# Exemple d'utilisation
image_path = "map.jpeg"
obstacles, zone_arrivee = detecter_obstacles_et_arrivee(image_path)

# Afficher les coordonnées des obstacles
for i, obstacle in enumerate(obstacles):
    print(f"Coordonnées obstacle {i + 1} :", obstacle)

# Afficher les coordonnées de la zone d'arrivée
print("Coordonnées zone d'arrivée :", zone_arrivee)


Coordonnées obstacle 1 : [(1300, 1025), (251, 1016), (247, 706), (1297, 715)]
Coordonnées obstacle 2 : [(1742, 3677), (1744, 3375), (2752, 3352), (2756, 3661)]
Coordonnées zone d'arrivée : [[(250, 2947), (1413, 2947), (1413, 1787), (250, 1787)]]
